In [7]:
import numpy as np
import pandas as pd
import opensmile


import pydub
import librosa
from scipy import signal
from mutagen.mp3 import MP3
import numpy as np
from numpy import sign, trapz
from teager_py import Teager
import statsmodels.api as sm
import wave

import warnings
warnings.filterwarnings('ignore')

In [8]:
# load path
path = pd.read_csv('/project/graziul/ra/team_ser/susas_path.csv')
path.drop(path.iloc[:, 0:1], inplace = True, axis = 1)
path = path[['Path','Arousal','Valence']]

In [9]:
path.head(2)

,Path,Arousal,Valence
0,/project/graziul/data/corpora/susas/speech/act...,High,Negative
1,/project/graziul/data/corpora/susas/speech/act...,High,Negative


In [10]:
# convert librosa to AudioSegment
def librosa_to_audiosegment(filename):
    y, sr = librosa.load(filename)
    # convert from float to uint16
    y = np.array(y * (1<<15), dtype=np.int16)
    audio_segment = pydub.AudioSegment(
        y.tobytes(), 
        frame_rate=sr,
        sample_width=y.dtype.itemsize, 
        channels=1
    )
    return audio_segment

In [11]:
# GeMAPS

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.GeMAPSv01b,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

def get_GeMAPS(path):
    GeMAPS = []
    for i in range(0,len(path)):
        filepath = path.loc[i,'Path']
        # audio,sr = librosa.load(filepath,sr=22050) # all speech tokens were sampled at a rate of 22050Hz
        audio = librosa_to_audiosegment(filepath) # convert librosa array to audiosegment
        sr = audio.frame_rate
        audio = np.array(audio.get_array_of_samples())
        df = smile.process_signal(audio,sr)
        GeMAPS.append(pd.DataFrame(df.mean(axis=0)).T)
        print('processing:',i)
    
    GeMAPS = pd.concat(GeMAPS).reset_index()
    return GeMAPS

In [17]:
GeMAPS_0105_features = pd.concat([path, GeMAPS_0105.iloc[:,1:]], axis=1)
GeMAPS_0105_features.head(2)

,Path,Arousal,Valence,Loudness_sma3,alphaRatio_sma3,hammarbergIndex_sma3,slope0-500_sma3,slope500-1500_sma3,F0semitoneFrom27.5Hz_sma3nz,jitterLocal_sma3nz,...,HNRdBACF_sma3nz,logRelF0-H1-H2_sma3nz,logRelF0-H1-A3_sma3nz,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3amplitudeLogRelF0_sma3nz
0,/project/graziul/data/corpora/susas/speech/act...,High,Negative,6.292282,-6.763290,18.529961,-0.086114,-0.002062,0.0,0.0,...,0.0,0.0,0.0,619.362793,730.045105,-201.0,1487.981445,-201.0,2418.518311,-201.0
1,/project/graziul/data/corpora/susas/speech/act...,High,Negative,6.322670,-6.638078,18.540451,-0.087078,-0.001739,0.0,0.0,...,0.0,0.0,0.0,627.472534,710.429443,-201.0,1501.422119,-201.0,2410.086670,-201.0


In [16]:
#GeMAPS_0105 = get_GeMAPS(path)

In [14]:
#GeMAPS_0105_features.to_csv('/project/graziul/ra/team_ser/GeMAPS_0105.csv')

In [18]:
# descriptive stat

In [ ]:
# feature extraction

In [46]:
# TEO
from teo_cb_auto_env import TeagorExtractor

t = TeagorExtractor()

def get_teo(sample):
    TEO = []
    for i in range(0,len(sample)):
        filepath = sample.iloc[i,0]
        teo_cb_auto_env = t.process_mp3(filepath,1)
        TEO.append(pd.DataFrame(pd.DataFrame(teo_cb_auto_env).mean(axis=0)).T)
    #print('processing:',i)
    TEO = pd.concat(TEO).reset_index()
    return TEO

In [ ]:
TE0_0105 = get_teo(sample)

In [ ]:
features = pd.concat([sample, GeMAPS_0105], axis=1)

In [ ]:
filepath = '/home/xuranzeng/SER/ProjectiveNum1.wav'

In [ ]:
def split_wav(self, filepath, wav_filepath):
        song = pydub.AudioSegment.from_wav(filepath)
        duration = song.duration_seconds * 1000
        # print(f"Wav file length: {duration}")
        length = 0
        self.audio_data_all = []
        while length + int(self.delta * 1000) <= duration:
            song_piece = song[length:length + int(self.delta * 1000)]
            self.audio_data_all.append(np.array(song_piece.get_array_of_samples()).astype(np.int64))
            length += int(self.delta * 1000)
        # print(len(self.audio_data_all))

In [ ]:
a = t.split_wav(filepath)